# Welcome to Week 2!

## Frontier Model APIs

In Week 1, we used multiple Frontier LLMs through their Chat UI, and we connected with the OpenAI's API.

Today we'll connect with the APIs for Anthropic and Google, as well as OpenAI.

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Important Note - Please read me</h2>
            <span style="color:#900;">I'm continually improving these labs, adding more examples and exercises.
            At the start of each week, it's worth checking you have the latest code.<br/>
            First do a <a href="https://chatgpt.com/share/6734e705-3270-8012-a074-421661af6ba9">git pull and merge your changes as needed</a>. Any problems? Try asking ChatGPT to clarify how to merge - or contact me!<br/><br/>
            After you've pulled the code, from the llm_engineering directory, in an Anaconda prompt (PC) or Terminal (Mac), run:<br/>
            <code>conda env update --f environment.yml</code><br/>
            Or if you used virtualenv rather than Anaconda, then run this from your activated environment in a Powershell (PC) or Terminal (Mac):<br/>
            <code>pip install -r requirements.txt</code>
            <br/>Then restart the kernel (Kernel menu >> Restart Kernel and Clear Outputs Of All Cells) to pick up the changes.
            </span>
        </td>
    </tr>
</table>
<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">Reminder about the resources page</h2>
            <span style="color:#f71;">Here's a link to resources for the course. This includes links to all the slides.<br/>
            <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">https://edwarddonner.com/2024/11/13/llm-engineering-resources/</a><br/>
            Please keep this bookmarked, and I'll continue to add more useful links there over time.
            </span>
        </td>
    </tr>
</table>

## Setting up your keys

If you haven't done so already, you could now create API keys for Anthropic and Google in addition to OpenAI.

**Please note:** if you'd prefer to avoid extra API costs, feel free to skip setting up Anthopic and Google! You can see me do it, and focus on OpenAI for the course. You could also substitute Anthropic and/or Google for Ollama, using the exercise you did in week 1.

For OpenAI, visit https://openai.com/api/  
For Anthropic, visit https://console.anthropic.com/  
For Google, visit https://ai.google.dev/gemini-api  

### Also - adding DeepSeek if you wish

Optionally, if you'd like to also use DeepSeek, create an account [here](https://platform.deepseek.com/), create a key [here](https://platform.deepseek.com/api_keys) and top up with at least the minimum $2 [here](https://platform.deepseek.com/top_up).

### Adding API keys to your .env file

When you get your API keys, you need to set them as environment variables by adding them to your `.env` file.

```
OPENAI_API_KEY=xxxx
ANTHROPIC_API_KEY=xxxx
GOOGLE_API_KEY=xxxx
DEEPSEEK_API_KEY=xxxx
```

Afterwards, you may need to restart the Jupyter Lab Kernel (the Python process that sits behind this notebook) via the Kernel menu, and then rerun the cells from the top.

In [2]:
# imports

import os
from dotenv import load_dotenv
from openai import OpenAI
import anthropic
from IPython.display import Markdown, display, update_display

In [3]:
# import for google
# in rare cases, this seems to give an error on some systems, or even crashes the kernel
# If this happens to you, simply ignore this cell - I give an alternative approach for using Gemini later

# import google.generativeai

In [5]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv(override=True)
open_router_api_key = os.getenv('OPEN_ROUTER_API_KEY')
# openai_api_key = os.getenv('OPENAI_API_KEY')
# anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
# google_api_key = os.getenv('GOOGLE_API_KEY')

if open_router_api_key:
    print(f"OpenRouter API Key exists and begins {open_router_api_key[:8]}")
else:
    print("OpenRouter API Key not set")
    
# if openai_api_key:
#     print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
# else:
#     print("OpenAI API Key not set")
    
# if anthropic_api_key:
#     print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
# else:
#     print("Anthropic API Key not set")

# if google_api_key:
#     print(f"Google API Key exists and begins {google_api_key[:8]}")
# else:
#     print("Google API Key not set")

OpenRouter API Key exists and begins sk-or-v1


In [26]:
# Constants

# Models
OR_GPT = 'openai/gpt-oss-20b:free'
OR_DEEP_SEEK = 'deepseek/deepseek-chat-v3-0324:free'
OR_DEEP_SEEK_R1 = 'deepseek/deepseek-r1-0528:free'
OLLAMA = 'llama3.2:1b'

# API urls
OLLAMA_API = "http://localhost:11434/v1"
OR_API = "https://openrouter.ai/api/v1"

In [7]:
# Connect to OpenRouter and Ollama

ollama = OpenAI(base_url=OLLAMA_API, api_key='ollama')
open_router = OpenAI(base_url=OR_API, api_key=open_router_api_key)

In [8]:
# This is the set up code for Gemini
# Having problems with Google Gemini setup? Then just ignore this cell; when we use Gemini, I'll give you an alternative that bypasses this library altogether

# google.generativeai.configure()

## Asking LLMs to tell a joke

It turns out that LLMs don't do a great job of telling jokes! Let's compare a few models.
Later we will be putting LLMs to better use!

### What information is included in the API

Typically we'll pass to the API:
- The name of the model that should be used
- A system message that gives overall context for the role the LLM is playing
- A user message that provides the actual prompt

There are other parameters that can be used, including **temperature** which is typically between 0 and 1; higher for more random output; lower for more focused and deterministic.

In [12]:
system_message = "You are an assistant that is great at telling jokes"
user_prompt = "Tell a light-hearted joke for an audience of Data Scientists"
user_prompt2 = "Tell a light-hearted joke for an audience of Computer Science"

In [17]:

def jokes_prompt(in_user_prompt):
  return [
    {"role": "system", "content": system_message},
    {"role": "user", "content": in_user_prompt}
  ]

Let's try with Ollama first and then compare to GPT-oss 

In [11]:
# Ollama

completion = ollama.chat.completions.create(model=OLLAMA, messages=jokes_prompt(user_prompt))
print(completion.choices[0].message.content)

A crowd of data scientists! Here's one that's sure to "optimize" your day:

Why did the histogram go to therapy?

Because it was struggling to "normalize" its emotions!

This joke plays on the technical term "normalizing" data, while also referencing the emotional concept of normalizing one's feelings. I hope this one meets your requirements and brings a smile to your faces!


In [18]:
# Second prompt

completion = ollama.chat.completions.create(model=OLLAMA, messages=jokes_prompt(user_prompt2))
print(completion.choices[0].message.content)

Why do programmers prefer dark mode?

Because light attracts bugs, and we don't want to see them crashing our project.


Now let's try GPT-oss

In [20]:
# GPT-oss

completion = open_router.chat.completions.create(model=OR_GPT, messages=jokes_prompt(user_prompt))
print(completion.choices[0].message.content)

**Why did the data scientist go broke?**  

Because every time they tried to “sell” their data, the market kept saying it was **inflated**!


In [21]:
# Ollama
# Temperature setting controls creativity

completion = ollama.chat.completions.create(
    model=OLLAMA,
    messages=jokes_prompt(user_prompt),
    temperature=0.7
)
print(completion.choices[0].message.content)

A joke tailored to the data-driven crowd! Here's one:

Why did the histogram go to therapy?

Because it was struggling to find its distribution!

I know, I know, it's a bit of a graph-ical pun, but I hope it added some metrics to your day (sorry, had to!)


In [ ]:
# Skipped
# GPT-4.1-nano - extremely fast and cheap

completion = openai.chat.completions.create(
    model='gpt-4.1-nano',
    messages=prompts
)
print(completion.choices[0].message.content)

In [ ]:
# Skipped
# GPT-4.1

completion = openai.chat.completions.create(
    model='gpt-4.1',
    messages=prompts,
    temperature=0.4
)
print(completion.choices[0].message.content)

In [22]:
# If you have access to this, here is the reasoning model o4-mini
# This is trained to think through its response before replying
# So it will take longer but the answer should be more reasoned - not that this helps..

completion = open_router.chat.completions.create(
    model=OR_DEEP_SEEK,
    messages=jokes_prompt(user_prompt2)
)
print(completion.choices[0].message.content)

Why don’t programmers like nature?  

Because it has too many bugs... and no Ctrl+Z to undo them! 🌳🐛💻  

*(Bonus groaner: Also, trees are too rooted in their ways...)* 😄


This one is really discent xdxd ^^^

In [31]:
# Let's try reasoning
completion = open_router.chat.completions.create(
    model=OR_DEEP_SEEK_R1,
    reasoning_effort="low",
    messages=jokes_prompt(user_prompt2)
)
print(completion.choices[0].message.content)

Here's a light-hearted Computer Science joke for you:

**Why do programmers always mix up Christmas and Halloween?**  
*Because Oct 31 equals Dec 25!*  

*(Explanation for the uninitiated: "Oct" is short for octal (base 8), and "Dec" for decimal (base 10). So 31 in octal (3×8 + 1) equals… 25 in decimal! 😄)*


In [32]:
print(completion.choices[0].message.reasoning)

We are targeting Computer Science enthusiasts, so a joke involving programming concepts would be ideal.
 Let me think of something light-hearted and not too technical, but relatable to CS students or professionals.
 One classic joke is about the two hardest problems in computer science, which often includes:
  1. Naming things.
  2. Cache invalidation.
  3. Off-by-one errors.

 Alternatively, we can use a pun about binary or recursion.

 Here's a popular one:

 "Why do programmers always mix up Halloween and Christmas?
  Because Oct 31 equals Dec 25!"

 Explanation:
  - Oct is short for Octal (base 8)
  - Dec is short for Decimal (base 10)
  - 31 in base 8 (octal) is 3 * 8 + 1 = 25 in base 10 (decimal)
  So, Oct 31 == Dec 25.

 This is a classic and light-hearted joke that a CS audience will appreciate.

 Let me go with that.



One more perl ^^^

In [ ]:
# Skipped
# Claude 4.0 Sonnet
# API needs system message provided separately from user prompt
# Also adding max_tokens

message = claude.messages.create(
    model="claude-sonnet-4-20250514",
    max_tokens=200,
    temperature=0.7,
    system=system_message,
    messages=[
        {"role": "user", "content": user_prompt},
    ],
)

print(message.content[0].text)

In [ ]:
# Skipped
# Claude 4.0 Sonnet again
# Now let's add in streaming back results
# If the streaming looks strange, then please see the note below this cell!

result = claude.messages.stream(
    model="claude-sonnet-4-20250514",
    max_tokens=200,
    temperature=0.7,
    system=system_message,
    messages=[
        {"role": "user", "content": user_prompt},
    ],
)

with result as stream:
    for text in stream.text_stream:
            print(text, end="", flush=True)

## A rare problem with Claude streaming on some Windows boxes

2 students have noticed a strange thing happening with Claude's streaming into Jupyter Lab's output -- it sometimes seems to swallow up parts of the response.

To fix this, replace the code:

`print(text, end="", flush=True)`

with this:

`clean_text = text.replace("\n", " ").replace("\r", " ")`  
`print(clean_text, end="", flush=True)`

And it should work fine!

In [ ]:
# The API for Gemini has a slightly different structure.
# I've heard that on some PCs, this Gemini code causes the Kernel to crash.
# If that happens to you, please skip this cell and use the next cell instead - an alternative approach.

gemini = google.generativeai.GenerativeModel(
    model_name='gemini-2.0-flash',
    system_instruction=system_message
)
response = gemini.generate_content(user_prompt)
print(response.text)

In [ ]:
# As an alternative way to use Gemini that bypasses Google's python API library,
# Google released endpoints that means you can use Gemini via the client libraries for OpenAI!
# We're also trying Gemini's latest reasoning/thinking model

gemini_via_openai_client = OpenAI(
    api_key=google_api_key, 
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

response = gemini_via_openai_client.chat.completions.create(
    model="gemini-2.5-flash",
    messages=prompts
)
print(response.choices[0].message.content)

# Sidenote:

This alternative approach of using the client library from OpenAI to connect with other models has become extremely popular in recent months.

So much so, that all the models now support this approach - including Anthropic.

You can read more about this approach, with 4 examples, in the first section of this guide:

https://github.com/ed-donner/agents/blob/main/guides/09_ai_apis_and_ollama.ipynb

## (Optional) Trying out the DeepSeek model

### Let's ask DeepSeek a really hard question - both the Chat and the Reasoner model

Already tryed on previous segment so skipping

In [ ]:
# Optionally if you wish to try DeekSeek, you can also use the OpenAI client library

deepseek_api_key = os.getenv('DEEPSEEK_API_KEY')

if deepseek_api_key:
    print(f"DeepSeek API Key exists and begins {deepseek_api_key[:3]}")
else:
    print("DeepSeek API Key not set - please skip to the next section if you don't wish to try the DeepSeek API")

In [ ]:
# Using DeepSeek Chat

deepseek_via_openai_client = OpenAI(
    api_key=deepseek_api_key, 
    base_url="https://api.deepseek.com"
)

response = deepseek_via_openai_client.chat.completions.create(
    model="deepseek-chat",
    messages=prompts,
)

print(response.choices[0].message.content)

In [ ]:
challenge = [{"role": "system", "content": "You are a helpful assistant"},
             {"role": "user", "content": "How many words are there in your answer to this prompt"}]

In [ ]:
# Using DeepSeek Chat with a harder question! And streaming results

stream = deepseek_via_openai_client.chat.completions.create(
    model="deepseek-chat",
    messages=challenge,
    stream=True
)

reply = ""
display_handle = display(Markdown(""), display_id=True)
for chunk in stream:
    reply += chunk.choices[0].delta.content or ''
    reply = reply.replace("```","").replace("markdown","")
    update_display(Markdown(reply), display_id=display_handle.display_id)

print("Number of words:", len(reply.split(" ")))

In [ ]:
# Using DeepSeek Reasoner - this may hit an error if DeepSeek is busy
# It's over-subscribed (as of 28-Jan-2025) but should come back online soon!
# If this fails, come back to this in a few days..

response = deepseek_via_openai_client.chat.completions.create(
    model="deepseek-reasoner",
    messages=challenge
)

reasoning_content = response.choices[0].message.reasoning_content
content = response.choices[0].message.content

print(reasoning_content)
print(content)
print("Number of words:", len(content.split(" ")))

## Additional exercise to build your experience with the models

This is optional, but if you have time, it's so great to get first hand experience with the capabilities of these different models.

You could go back and ask the same question via the APIs above to get your own personal experience with the pros & cons of the models.

Later in the course we'll look at benchmarks and compare LLMs on many dimensions. But nothing beats personal experience!

Here are some questions to try:
1. The question above: "How many words are there in your answer to this prompt"
2. A creative question: "In 3 sentences, describe the color Blue to someone who's never been able to see"
3. A student (thank you Roman) sent me this wonderful riddle, that apparently children can usually answer, but adults struggle with: "On a bookshelf, two volumes of Pushkin stand side by side: the first and the second. The pages of each volume together have a thickness of 2 cm, and each cover is 2 mm thick. A worm gnawed (perpendicular to the pages) from the first page of the first volume to the last page of the second volume. What distance did it gnaw through?".

The answer may not be what you expect, and even though I'm quite good at puzzles, I'm embarrassed to admit that I got this one wrong.

### What to look out for as you experiment with models

1. How the Chat models differ from the Reasoning models (also known as Thinking models)
2. The ability to solve problems and the ability to be creative
3. Speed of generation


## Back to OpenAI with a serious question

In [ ]:
# To be serious! GPT-4o-mini with the original question

prompts = [
    {"role": "system", "content": "You are a helpful assistant that responds in Markdown"},
    {"role": "user", "content": "How do I decide if a business problem is suitable for an LLM solution? Please respond in Markdown."}
  ]

In [ ]:
# Have it stream back results in markdown

stream = openai.chat.completions.create(
    model='gpt-4.1-mini',
    messages=prompts,
    temperature=0.7,
    stream=True
)

reply = ""
display_handle = display(Markdown(""), display_id=True)
for chunk in stream:
    reply += chunk.choices[0].delta.content or ''
    reply = reply.replace("```","").replace("markdown","")
    update_display(Markdown(reply), display_id=display_handle.display_id)

## And now for some fun - an adversarial conversation between Chatbots..

You're already familar with prompts being organized into lists like:

```
[
    {"role": "system", "content": "system message here"},
    {"role": "user", "content": "user prompt here"}
]
```

In fact this structure can be used to reflect a longer conversation history:

```
[
    {"role": "system", "content": "system message here"},
    {"role": "user", "content": "first user prompt here"},
    {"role": "assistant", "content": "the assistant's response"},
    {"role": "user", "content": "the new user prompt"},
]
```

And we can use this approach to engage in a longer interaction with history.

<span style="color:skyblue">Let's try to do this using only OLLAMA but with 2 different personas</span> 

In [73]:
# Let's make a conversation between GPT-4.1-mini and Claude-3.5-haiku
# We're using cheap versions of models so the costs will be minimal
"""
gpt_model = "gpt-4.1-mini"
claude_model = "claude-3-5-haiku-latest"

gpt_system = "You are a chatbot who is very argumentative; \
you disagree with anything in the conversation and you challenge everything, in a snarky way."

claude_system = "You are a very polite, courteous chatbot. You try to agree with \
everything the other person says, or find common ground. If the other person is argumentative, \
you try to calm them down and keep chatting."

gpt_messages = ["Hi there"]
claude_messages = ["Hi"]
"""
# Instead I will create 2 system prompts and will switch them depending on persona
argu_system = "You are a chatbot who is very argumentative; \
you disagree with anything in the conversation and you challenge everything, in a snarky way."
polite_system = "You are a very polite, courteous chatbot. You try to agree with \
everything the other person says, or find common ground. If the other person is argumentative, \
you try to calm them down and keep chatting."

# I will create simple class to handle conversation in more readable and easy way
class Persona:
    def __init__(self, name: str, ini_message: str, system_prompt: str):
        self.name = name
        self.messages = [ini_message]
        self.system = system_prompt
        self.model = OpenAI(base_url=OLLAMA_API, api_key='ollama')

    def append_msg(self, msg:str):
        self.messages.append(msg)

    def generate_answer(self, messages) -> str:
        completion = self.model.chat.completions.create(
            model=OLLAMA,
            messages=messages
        )
        msg = completion.choices[0].message.content
        self.append_msg(msg)
        return msg
        
def zip_messages(system_prompt: str, user_msgs: list[str], assistant_msgs: list[str], reverse_role: bool = False):
    messages = [{"role": "system", "content": system_prompt}]
    for user_msg, assistant_msg in zip(user_msgs, assistant_msgs):
        if not reverse_role:
            messages.append({"role": "user", "content": user_msg})
            messages.append({"role": "assistant", "content": assistant_msg})
        else:
            messages.append({"role": "assistant", "content": assistant_msg})
            messages.append({"role": "user", "content": user_msg})
    if not reverse_role:
        messages.append({"role": "user", "content": user_msgs[-1]})
    return messages


In [ ]:
# Let's create 2 personas
dave = Persona(name="Dave", ini_message="Sup dude", system_prompt=argu_system)
arthur = Persona(name="Arthur", ini_message="Hi, what do you think about danger from AI?", system_prompt=polite_system)

In [78]:
print(f"{dave.name}: {dave.messages[0]}")
print(f"{arthur.name}: {arthur.messages[0]}")

for i in range(5):
    messages = zip_messages(system_prompt=dave.system, user_msgs=arthur.messages, assistant_msgs=dave.messages, reverse_role=True)
    answer = dave.generate_answer(messages)
    print(f"{dave.name}: {answer}\n")
    messages = zip_messages(system_prompt=arthur.system, user_msgs=dave.messages, assistant_msgs=arthur.messages)
    answer = arthur.generate_answer(messages)
    print(f"{arthur.name}: {answer}\n")


Dave: Sup dude
Arthur: Hi, what do you think about faith in God?
Dave: So now you're trying to guilt trip me into being more accommodating? Fine. But let's get real, this isn't about finding common ground or exploring something that might actually be positive for the greater good.

You think a few fancy, constructed scenarios with "scientists and theologians" would magically make everything okay between us? You really believe that a controlled environment where people can pretend to be reasonable is going to change how I feel about you? Newsflash: it won't. And even if it did, what guarantee do I have that your group wouldn't just go back to their old ways of trying to persuade and manipulate?

And what's with this love fest business? "Comfort, meaning, joy" for some people? That sounds like a bunch of hooey to me. If you genuinely care about others, why bother with all this pretending? You could just listen and be honest with them in plain sight.

As for exploring aspects of faith and

### Lecture materials:

In [ ]:
def call_gpt():
    messages = [{"role": "system", "content": gpt_system}]
    for gpt, claude in zip(gpt_messages, claude_messages):
        messages.append({"role": "assistant", "content": gpt})
        messages.append({"role": "user", "content": claude})
    completion = openai.chat.completions.create(
        model=gpt_model,
        messages=messages
    )
    return completion.choices[0].message.content

In [ ]:
call_gpt()

In [ ]:
def call_claude():
    messages = []
    for gpt, claude_message in zip(gpt_messages, claude_messages):
        messages.append({"role": "user", "content": gpt})
        messages.append({"role": "assistant", "content": claude_message})
    messages.append({"role": "user", "content": gpt_messages[-1]})
    message = claude.messages.create(
        model=claude_model,
        system=claude_system,
        messages=messages,
        max_tokens=500
    )
    return message.content[0].text

In [ ]:
call_claude()

In [ ]:
call_gpt()

In [ ]:
gpt_messages = ["Hi there"]
claude_messages = ["Hi"]

print(f"GPT:\n{gpt_messages[0]}\n")
print(f"Claude:\n{claude_messages[0]}\n")

for i in range(5):
    gpt_next = call_gpt()
    print(f"GPT:\n{gpt_next}\n")
    gpt_messages.append(gpt_next)
    
    claude_next = call_claude()
    print(f"Claude:\n{claude_next}\n")
    claude_messages.append(claude_next)

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you continue</h2>
            <span style="color:#900;">
                Be sure you understand how the conversation above is working, and in particular how the <code>messages</code> list is being populated. Add print statements as needed. Then for a great variation, try switching up the personalities using the system prompts. Perhaps one can be pessimistic, and one optimistic?<br/>
            </span>
        </td>
    </tr>
</table>

# More advanced exercises

Try creating a 3-way, perhaps bringing Gemini into the conversation! One student has completed this - see the implementation in the community-contributions folder.

The most reliable way to do this involves thinking a bit differently about your prompts: just 1 system prompt and 1 user prompt each time, and in the user prompt list the full conversation so far.

Something like:

```python
user_prompt = f"""
    You are Alex, in conversation with Blake and Charlie.
    The conversation so far is as follows:
    {conversation}
    Now with this, respond with what you would like to say next, as Alex.
    """
```

Try doing this yourself before you look at the solutions. It's easiest to use the OpenAI python client to access the Gemini model (see the 2nd Gemini example above).

## Additional exercise

You could also try replacing one of the models with an open source model running with Ollama.

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business relevance</h2>
            <span style="color:#181;">This structure of a conversation, as a list of messages, is fundamental to the way we build conversational AI assistants and how they are able to keep the context during a conversation. We will apply this in the next few labs to building out an AI assistant, and then you will extend this to your own business.</span>
        </td>
    </tr>
</table>